In [5]:
import torch
from torch import nn
import classification_ModelNet40.models as models
import torch.backends.cudnn as cudnn
from classification_ModelNet40.models import pointMLP

# from cell_dataset import PointCloudDatasetAllBoth
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
from foldingnet import ReconstructionNet, ChamferLoss
from dataset import (
    PointCloudDatasetAllBoth,
    PointCloudDatasetAllBothNotSpec,
    PointCloudDatasetAll,
)
import argparse
import os
from tearing.folding_decoder import FoldingNetBasicDecoder
import torch
from torch import nn
import classification_ModelNet40.models as models
import torch.backends.cudnn as cudnn
from classification_ScanObjectNN.models import pointMLPElite

# from cell_dataset import PointCloudDatasetAllBoth
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
from foldingnet import ReconstructionNet, ChamferLoss
from dataset import (
    PointCloudDatasetAllBoth,
    PointCloudDatasetAllBothNotSpec,
    PointCloudDatasetAllBothNotSpec1024,
)
import argparse
import os
from tearing.folding_decoder import FoldingNetBasicDecoder


class MLPAutoencoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(MLPAutoencoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, x):
        embedding = self.encoder(x)
        output, grid = self.decoder(embedding)
        return output, embedding, grid


def create_dir_if_not_exist(path):
    if not os.path.exists(path):
        os.makedirs(path)




df = "/home/mvries/Documents/Datasets/OPM/SingleCellFromNathan_17122021/all_cell_data.csv"
root_dir = "/home/mvries/Documents/Datasets/OPM/SingleCellFromNathan_17122021/"
output_path = "./"
num_epochs = 250
pmlp_ckpt_path = "best_checkpoint.pth"
fold_ckpt_path = "/home/mvries/Documents/GitHub/FoldingNetNew/nets/FoldingNetNew_50feats_planeshape_foldingdecoder_trainallTrue_centringonlyTrue_train_bothTrue_003.pt"
pmlp_folding_ckpt_path = "./pointmlpelite_foldingTearingVersion_autoencoder_allparams2048.pt"

net = pointMLPElite(ponts=2048)
device = "cuda"
net = net.to(device)
if device == "cuda":
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True
new_embedding = nn.Linear(in_features=256, out_features=50, bias=True)
net.module.classifier[8] = new_embedding
net.module.classifier[8].weight.requires_grad = True
net.module.classifier[8].bias.requires_grad = True
print(net.module.classifier)

    # to_eval = (
    #         "ReconstructionNet"
    #         + "("
    #         + "'{0}'".format("dgcnn_cls")
    #         + ", num_clusters=5, num_features = 50, shape='plane')"
    # )
    # decoder = eval(to_eval)
decoder = FoldingNetBasicDecoder(num_features=50, num_clusters=10)
model = MLPAutoencoder(encoder=net.module, decoder=decoder).cuda()
checkpoint = torch.load(pmlp_folding_ckpt_path)
model.load_state_dict(checkpoint["model_state_dict"])
print(checkpoint["loss"])

data = torch.rand(2, 3, 2048).cuda()
print("===> testing pointMLP ...")
out, embedding, _ = model(data)
print(out.shape)
print(embedding.shape)
criterion = ChamferLoss()

print("Extrancting cell")

dataset = PointCloudDatasetAllBothNotSpec(
        df,
        root_dir,
        transform=None,
        img_size=400,
        target_transform=True,
        centring_only=True,
        cell_component="cell",
    )

dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

from tqdm import tqdm

inputs_test = []
outputs_test = []
features_test = []
embeddings_test = []
clusterings_test = []
labels_test = []
serial_numbers = []
model.eval()
loss = 0
for data in tqdm(dataloader):
    with torch.no_grad():
        pts, lab, serial_num = data

        labels_test.append(lab.detach().numpy())
        inputs = pts.to(device)
        output, embedding, _ = model(inputs.permute(0, 2, 1))
        inputs_test.append(torch.squeeze(inputs).cpu().detach().numpy())
        outputs_test.append(torch.squeeze(output).cpu().detach().numpy())
        embeddings_test.append(torch.squeeze(embedding).cpu().detach().numpy())
        serial_numbers.append(serial_num)
        loss += criterion(inputs, output)

print(loss / len(dataloader))
folding_data = pd.DataFrame(np.asarray(embeddings_test))
folding_data["serialNumber"] = np.asarray(serial_numbers)
all_data = pd.read_csv(df)
all_data_labels = all_data[
        [
            "serialNumber",
            "Treatment",
            "Proximal",
            "nucleusCoverslipDistance",
            "erkRatio",
            "erkIntensityNucleus",
            "erkIntensityCell",
        ]
    ]
folding_data_new = folding_data.join(
        all_data_labels.set_index("serialNumber"), on="serialNumber"
    )

folding_data_new.to_csv(
        "/home/mvries/Documents/Datasets/OPM/SingleCellFromNathan_17122021/"
        "pointmlpfolding_50_cell_all_centring.csv"
    )

Sequential(
  (0): Linear(in_features=256, out_features=512, bias=True)
  (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): Dropout(p=0.5, inplace=False)
  (4): Linear(in_features=512, out_features=256, bias=True)
  (5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU(inplace=True)
  (7): Dropout(p=0.5, inplace=False)
  (8): Linear(in_features=256, out_features=50, bias=True)
)
34.91719532491299
===> testing pointMLP ...
torch.Size([2, 2025, 3])
torch.Size([2, 50])
Extrancting cell


  0%|                                                                                                          | 30/70167 [00:01<1:12:30, 16.12it/s]


KeyboardInterrupt: 

In [7]:
number = 30
points = outputs_test[number]
print(points)
points_nuc = inputs_test[number]
print(points_nuc.shape)

# v = pptk.viewer(points)
# draw_geometries([])

# xyz = pptk.rand(100, 3)
# v = pptk.viewer(xyz)
# v.capture('screenshot.png')
# v.clear()
# v.close()

# import numpy as np
# import pyvista as pv
# from pyvista import examples
# point_cloud = pv.PolyData(points)
# point_cloud
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import plotly.express as px


# fig = plt.figure(figsize=(5, 5))
# ax = plt.axes(projection='3d')

# ax.scatter(points[:,0], points[:,1], points[:,2], c='y', marker='o', s=10)

# fig = plt.figure(figsize=(5, 5))
# ax = plt.axes(projection='3d')

# ax.scatter(points_nuc[:,0], points_nuc[:,1], c='y', marker='o', s=10)

data = pd.DataFrame(points, columns=['x','y', 'z'])
fig = px.scatter_3d(data, x="x", y="y", z='z',
               color_discrete_sequence=['red']
                )

fig.update_traces(marker=dict(size=2),
                  selector=dict(mode='markers'))
fig.update_layout(
width = 600,
height = 600
)

fig.show()

points = inputs_test[number]
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt



data = pd.DataFrame(points, columns=['x','y', 'z'])
fig = px.scatter_3d(data, x="x", y="y", z='z',
               color_discrete_sequence=['red']
                )

fig.update_traces(marker=dict(size=2),
                  selector=dict(mode='markers'))
fig.update_layout(
            width = 600,
            height = 600
            )
# fig.update_layout(legend=dict(
#         yanchor="top",
#         y=0.99,
#         xanchor="right",
#         x=0.99,
#         font=dict(
#             size=15
#         ),
#         ))
fig.show()
print(serial_numbers[number])

[[-0.26375526  0.02956996  0.03797984]
 [-0.2623211   0.02746109  0.03900588]
 [-0.26109374  0.0248281   0.03947152]
 ...
 [ 0.35140395 -0.80134994  0.00902081]
 [ 0.32336542 -0.6827063  -0.17466575]
 [ 0.29136762 -0.5124236  -0.34578127]]
(2048, 3)


('0001_0031_accelerator_20210315_bakal01_erk_main_21-03-15_12-37-27',)
